In [2]:
# # This mounts your Google Drive to the Colab VM.
# from google.colab import drive
# drive.mount('/content/drive')

# # TODO: Enter the foldername in your Drive where you have saved the unzipped
# # assignment folder, e.g. 'cs231n/assignments/assignment1/'
# FOLDERNAME = None
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# # Now that we've mounted your Drive, this ensures that
# # the Python interpreter of the Colab VM can load
# # python files from within it.
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# # This downloads the CIFAR-10 dataset to your Drive
# # if it doesn't already exist.
# %cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
# !bash get_datasets.sh
# %cd /content/drive/My\ Drive/$FOLDERNAME

# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [3]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside `cs231n/classifiers/softmax.py`.


In [5]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.354988
sanity check: 2.302585


**Inline Question 1**

Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

$\color{blue}{\textit Your Answer:}$ because the weight is generated randomly, softmax is making a random guess. for a 10-class classification, a random guess has the accuracy of 10%, so $ \frac{e^{f_{y_i}}}{\sum_j{e^{f_j}}} = 0.1$, hence the loss is $-np.log(0.1)$



In [6]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -2.728362 analytic: -2.728362, relative error: 2.028532e-08
numerical: -2.802716 analytic: -2.802716, relative error: 2.308090e-08
numerical: -0.278885 analytic: -0.278885, relative error: 8.841593e-08
numerical: 0.686268 analytic: 0.686268, relative error: 6.825303e-08
numerical: 1.480529 analytic: 1.480529, relative error: 1.401602e-08
numerical: 2.269317 analytic: 2.269317, relative error: 4.967644e-09
numerical: 1.049816 analytic: 1.049816, relative error: 1.606297e-08
numerical: 0.541521 analytic: 0.541521, relative error: 2.606936e-08
numerical: 0.101006 analytic: 0.101006, relative error: 2.258519e-07
numerical: -0.016948 analytic: -0.016948, relative error: 8.225116e-07
numerical: -0.465638 analytic: -0.465639, relative error: 1.391985e-07
numerical: -2.019673 analytic: -2.019673, relative error: 7.208695e-09
numerical: -0.193339 analytic: -0.193339, relative error: 2.677409e-07
numerical: 1.188104 analytic: 1.188104, relative error: 1.279920e-08
numerical: -0.409071

In [12]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
w = np.array([[1,2,3],[4,5,6], [7,8,9], [10,11,12]], dtype = np.float64)  # (D, C)
print("w shape:", w.shape)
x = np.array([[0,0,0,0], [1, 1, 1, 1], [2, 2,2,2], [3,3,3,3], [4, 4,4,4] ])   # (N, D)
print("x shape:", x.shape)
y = np.array([0, 0, 1, 1, 2])
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(w, x, y, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(w, x, y, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

w shape: (4, 3)
x shape: (5, 4)
naive loss: 5.826737e+00 computed in 0.000498s
scores: [[  0.   0.   0.]
 [ 22.  26.  30.]
 [ 44.  52.  60.]
 [ 66.  78.  90.]
 [ 88. 104. 120.]]
exp scores: [[1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [3.35462628e-04 1.83156389e-02 1.00000000e+00]
 [1.12535175e-07 3.35462628e-04 1.00000000e+00]
 [3.77513454e-11 6.14421235e-06 1.00000000e+00]
 [1.26641655e-14 1.12535175e-07 1.00000000e+00]]
sums [3.         1.0186511  1.00033558 1.00000614 1.00000011]
loss 5.826736673379901
ds [[3.33333333e-01 3.33333333e-01 3.33333333e-01]
 [3.29320439e-04 1.79802867e-02 9.81690393e-01]
 [1.12497423e-07 3.35350093e-04 9.99664537e-01]
 [3.77511135e-11 6.14417460e-06 9.99993856e-01]
 [1.26641641e-14 1.12535162e-07 9.99999887e-01]]
Y: [0 0 1 1 2]
ds adjust y [[-6.66666667e-01  3.33333333e-01  3.33333333e-01]
 [-9.99670680e-01  1.79802867e-02  9.81690393e-01]
 [ 1.12497423e-07 -9.99664650e-01  9.99664537e-01]
 [ 3.77511135e-11 -9.99993856e-01  9.99993856e-01]
 [ 1.2664

In [38]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.347950e+00 computed in 0.056820s
scores: [[-5.61730743e-01  2.37557133e-01 -9.17190169e-02 ... -2.51791338e-02
   1.48088052e-01 -1.30908878e-01]
 [ 2.28900746e-01  2.55225445e-02 -2.15470551e-03 ... -4.37336308e-01
  -5.21714579e-01  1.86590750e-01]
 [-1.98026249e-01 -7.22022409e-02 -4.11763131e-02 ... -6.63122374e-02
  -8.43612433e-02 -6.54088382e-02]
 ...
 [-1.21466248e-01 -1.23308360e-01 -4.17778482e-01 ... -5.66066641e-02
  -7.20916418e-01 -1.38342049e-01]
 [-5.55613907e-02  3.56670821e-01  3.56044218e-01 ...  9.77954393e-01
   2.02415372e-01 -7.60001974e-02]
 [ 1.49482649e-01 -3.70439763e-01  1.04907563e-02 ... -2.26691817e-01
  -2.66929988e-01 -1.96240197e-04]]
exp scores: [[0.28631841 0.63675973 0.45811317 ... 0.489633   0.58226363 0.44050702]
 [1.         0.81596958 0.79369545 ... 0.51363774 0.47207598 0.95857258]
 [0.66021051 0.74873322 0.77232749 ... 0.75315628 0.7396845  0.75383699]
 ...
 [0.3124396  0.31186458 0.2323161  ... 0.33337593 0.17156479 0.30721117]


In [24]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.

from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

best validation accuracy achieved during cross-validation: -1.000000


In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question 2** - *True or False*

Suppose the overall training loss is defined as the sum of the per-datapoint loss over all training examples. It is possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

$\color{blue}{\textit Your Answer:}$


$\color{blue}{\textit Your Explanation:}$



In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])